In [1]:
# 오늘기준으로 1년치 주가 csv 파일로 로컬에 저장

import yfinance as yf
from datetime import datetime, timedelta
datetime.now().date()
df = yf.download('005930.KS',datetime.now().date()-timedelta(365),datetime.now().date())
df.to_csv('samsung.csv')

[*********************100%%**********************]  1 of 1 completed


In [15]:
import findspark
findspark.init()

In [24]:
from pyspark.sql import (
    functions as f,
    Row,
    SparkSession,
    types as t
)

from pyspark.sql.functions import col, asc, desc

In [3]:

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate() 

In [27]:
csv_file_path = "samsung.csv"
df = spark.read.option("header","true") \
    .option("inferSchema","true") \
    .csv(csv_file_path)

In [68]:
# data = df.filter(df.Date>='2023-01-01')
df.count()

248

In [150]:
# ROW 추가
columns = ["Date", "Close", "expect"]

new_row = spark.createDataFrame([('2023-08-25', 68200.0, 68200.0)], columns)
expect_data = expect_data.union(new_row)
expect_data.show()

+----------+-------+-------+
|      Date|  Close| expect|
+----------+-------+-------+
|2023-01-02|55500.0|55500.0|
|2023-01-03|55400.0|55400.0|
|2023-01-04|57800.0|57800.0|
|2023-01-05|58200.0|58200.0|
|2023-01-06|59000.0|59000.0|
|2023-01-09|60700.0|60700.0|
|2023-01-10|60400.0|60400.0|
|2023-01-11|60500.0|60500.0|
|2023-01-12|60500.0|60500.0|
|2023-01-13|60800.0|60800.0|
|2023-01-16|61100.0|61100.0|
|2023-01-17|61000.0|61000.0|
|2023-01-18|60400.0|60400.0|
|2023-01-19|61500.0|61500.0|
|2023-01-20|61800.0|61800.0|
|2023-01-25|63300.0|63300.0|
|2023-01-26|63800.0|63800.0|
|2023-01-27|64600.0|64600.0|
|2023-01-30|63300.0|63300.0|
|2023-01-31|61400.0|61400.0|
+----------+-------+-------+
only showing top 20 rows



In [117]:

recent_data = df.withColumn("expect",f.col("Close")).select("Date","Close","expect")
recent_data.show()

+----------+-------+-------+
|      Date|  Close| expect|
+----------+-------+-------+
|2022-08-26|60000.0|60000.0|
|2022-08-29|58600.0|58600.0|
|2022-08-30|58800.0|58800.0|
|2022-08-31|59700.0|59700.0|
|2022-09-01|58400.0|58400.0|
|2022-09-02|57500.0|57500.0|
|2022-09-05|57100.0|57100.0|
|2022-09-06|57100.0|57100.0|
|2022-09-07|56000.0|56000.0|
|2022-09-08|55600.0|55600.0|
|2022-09-13|58100.0|58100.0|
|2022-09-14|56800.0|56800.0|
|2022-09-15|56000.0|56000.0|
|2022-09-16|56200.0|56200.0|
|2022-09-19|56400.0|56400.0|
|2022-09-20|55800.0|55800.0|
|2022-09-21|55300.0|55300.0|
|2022-09-22|54400.0|54400.0|
|2022-09-23|54500.0|54500.0|
|2022-09-26|53900.0|53900.0|
+----------+-------+-------+
only showing top 20 rows



In [126]:
# 최종 종가
last_close = recent_data.tail(1)[0][1]
print(last_close)

67100.0


In [7]:
# # 마지막 날짜를 일수로 변환
# from pyspark.sql.functions import date_add, date_sub, col

# today = expect_data.select(f.dayofyear('Date')).tail(1)[0][0]
# print(today)

# expect_data.select(date_add(col("Date"),today)).show()

236
+-------------------+
|date_add(Date, 236)|
+-------------------+
|         2023-08-26|
|         2023-08-27|
|         2023-08-28|
|         2023-08-29|
|         2023-08-30|
|         2023-09-02|
|         2023-09-03|
|         2023-09-04|
|         2023-09-05|
|         2023-09-06|
|         2023-09-09|
|         2023-09-10|
|         2023-09-11|
|         2023-09-12|
|         2023-09-13|
|         2023-09-18|
|         2023-09-19|
|         2023-09-20|
|         2023-09-23|
|         2023-09-24|
+-------------------+
only showing top 20 rows



In [118]:
# 1년 예측이지만 휴일 거래일 제외하면 248일
import pandas as pd
from pyspark.sql.functions import current_date, current_timestamp
from datetime import datetime, timedelta

now = datetime.now()
recent_day_count = recent_data.count()
print("현재 날짜 : ", datetime.now().date()+timedelta(days=1))

datelist = []

for x in range(recent_day_count) :
    datelist.append(datetime.now().date()+timedelta(days=x))
df_front_1year = pd.DataFrame({'Date': datelist})
ex_df_front_1year = pd.DataFrame({'Date': datelist})
print(df_front_1year)

현재 날짜 :  2023-08-28
           Date
0    2023-08-27
1    2023-08-28
2    2023-08-29
3    2023-08-30
4    2023-08-31
..          ...
243  2024-04-26
244  2024-04-27
245  2024-04-28
246  2024-04-29
247  2024-04-30

[248 rows x 1 columns]


In [119]:
# 앞으로 1년(248 거래일) 동안의 주가 예측 
import pandas as pd
# 1년간 주가의 평균
pd_recent_data = recent_data.toPandas()
close_avg = pd_recent_data['Close'].mean()
close_max = pd_recent_data['Close'].max()
close_min = pd_recent_data['Close'].min()
print(close_avg)
print(close_max)
print(close_min)

difference = pd_recent_data['Close'].apply(lambda x: x-close_avg)

print(difference)

63198.790322580644
73400.0
52600.0
0     -3198.790323
1     -4598.790323
2     -4398.790323
3     -3498.790323
4     -4798.790323
          ...     
243    3401.209677
244    3401.209677
245    3901.209677
246    5001.209677
247    3901.209677
Name: Close, Length: 248, dtype: float64


In [120]:
from pyspark.sql.types import *
df_difference = spark.createDataFrame(difference,'float')
# schema_income.collect()
pd_difference = df_difference.toPandas()
pd_difference = df_front_1year.insert(1,'difference',pd_difference)
pd_difference = df_front_1year
print(pd_difference)

           Date   difference
0    2023-08-27 -3198.790283
1    2023-08-28 -4598.790527
2    2023-08-29 -4398.790527
3    2023-08-30 -3498.790283
4    2023-08-31 -4798.790527
..          ...          ...
243  2024-04-26  3401.209717
244  2024-04-27  3401.209717
245  2024-04-28  3901.209717
246  2024-04-29  5001.209473
247  2024-04-30  3901.209717

[248 rows x 2 columns]


In [121]:
# pd_difference 날짜 기준으로 내림차순 정렬
df_sorted_by_values = pd_difference.sort_values(by='Date' ,ascending=False)
# print(df_sorted_by_values['difference'])
df_sorted = spark.createDataFrame(df_sorted_by_values['difference'],'float')
pd_sorted = df_sorted.toPandas()
ex_df_front_1year.insert(1,'difference',pd_sorted)

print(ex_df_front_1year)


           Date   difference
0    2023-08-27  3901.209717
1    2023-08-28  5001.209473
2    2023-08-29  3901.209717
3    2023-08-30  3401.209717
4    2023-08-31  3401.209717
..          ...          ...
243  2024-04-26 -4798.790527
244  2024-04-27 -3498.790283
245  2024-04-28 -4398.790527
246  2024-04-29 -4598.790527
247  2024-04-30 -3198.790283

[248 rows x 2 columns]


In [127]:
#예측 가격 = 평균가-pd_difference['difference']
ex_df_front_1year.insert(2,'expect',last_close-ex_df_front_1year['difference'])


In [128]:
print(ex_df_front_1year)

           Date   difference        expect
0    2023-08-27  3901.209717  63198.789062
1    2023-08-28  5001.209473  62098.789062
2    2023-08-29  3901.209717  63198.789062
3    2023-08-30  3401.209717  63698.789062
4    2023-08-31  3401.209717  63698.789062
..          ...          ...           ...
243  2024-04-26 -4798.790527  71898.789062
244  2024-04-27 -3498.790283  70598.789062
245  2024-04-28 -4398.790527  71498.789062
246  2024-04-29 -4598.790527  71698.789062
247  2024-04-30 -3198.790283  70298.789062

[248 rows x 3 columns]


In [129]:
# 예측값 csv파일로 저장
ex_df_front_1year.to_csv("expect.csv")